$ \frac{1}{4} \sum_{m<n} J_{HS}(m-n)\, \vec{\sigma}_m\cdot\vec{\sigma}_n $ where $J_{HS}(m-n) = \frac{\\pi^2}{N^2 \sin^2 ((m-n)\pi / N)}$, $N$ is number of sites. Use periodic boundary conditions

In [1]:
import numpy as np
import netket as nk
import flax.linen  as nn
from netket.operator.spin import sigmax, sigmay, sigmaz
import matplotlib.pyplot as plt
from src.hamiltonians import wrap_index

In [2]:
def haldane_shastry_model(N):
    hi = nk.hilbert.Spin(s=1/2, N=N)
    sum = 0
    for n in range(N):
        for m in range(n):
            hs = (np.pi**2) / (N**2 * (np.sin((wrap_index(N, n, m))*np.pi/N)**2))
            x = sigmax(hi, m) * sigmax(hi, n)
            y = sigmay(hi, m) * sigmay(hi, n)
            z = sigmaz(hi, m) * sigmaz(hi, n)
            sum += hs * (x+y+z)
    return sum / 4

## Training
Note that the number of hidden features, the samples, and the number of optimization epochs increases with system size.

In [4]:
nqs_en_x = []
nqs_en_y = []
analytical_en_x = []
analytical_en_y = []

In [16]:
for N in range(3, 29):
    if not N % 2 == 0:
        continue
    
    print(f'running: N={N}')

    hs = haldane_shastry_model(N)
    rbm = nk.models.RBM(alpha=1*max(1, N/10), param_dtype=complex, kernel_init=nn.initializers.normal(stddev=0.01))

    hi = nk.hilbert.Spin(s=1/2, N=N)
    sampler = nk.sampler.MetropolisLocal(hi)

    vstate = nk.vqs.MCState(sampler, model=rbm, n_samples=512 + 16 * N)
    optimizer = nk.optimizer.Sgd(learning_rate=0.05)
    preconditioner = nk.optimizer.SR(diag_shift=0.1, holomorphic=False)
    vmc = nk.driver.VMC(hs, optimizer, variational_state=vstate, preconditioner=preconditioner)

    true_energy = -np.pi**2 / 24 * (N + 5 / N) 
    print('expecting energy', true_energy)

    log = nk.logging.RuntimeLog()
    vmc.run(n_iter=250 + 15 * N, out=log)
    opt_energy = vstate.expect(hs)

    nqs_en_x.append(N)
    analytical_en_x.append(N)
    nqs_en_y.append(opt_energy.mean.real)
    analytical_en_y.append(true_energy)

running: N=4
expecting energy -2.158975962738297


100%|██████████| 310/310 [00:01<00:00, 207.95it/s, Energy=-2.158950-0.000007j ± 0.000036 [σ²=0.000001, R̂=1.3815]]    


running: N=6
expecting energy -2.8100956975323865


100%|██████████| 340/340 [00:03<00:00, 97.98it/s, Energy=-2.8166-0.0065j ± 0.0012 [σ²=0.0009, R̂=1.3698]]     


running: N=8
expecting energy -3.546889081641488


...

100%|██████████| 610/610 [04:29<00:00,  2.27it/s, Energy=-9.3466-0.0314j ± 0.0050 [σ²=0.0225, R̂=1.1965]]


running: N=26
expecting energy -10.771154803111944


100%|██████████| 640/640 [06:18<00:00,  1.69it/s, Energy=-10.7601-0.0143j ± 0.0066 [σ²=0.0403, R̂=1.2295]]


running: N=28
expecting energy -11.587973024493309


100%|██████████| 670/670 [08:45<00:00,  1.27it/s, Energy=-10.1374+0.0303j ± 0.0060 [σ²=0.0345, R̂=1.2066]]


In [17]:
np.savetxt("../data/hs_outputs_2.txt", np.column_stack((nqs_en_x, nqs_en_y, analytical_en_x, analytical_en_y)))